In [1]:
import pickle
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np

%matplotlib inline

### Helper Functions

The first two are for loading and storing dictionaries to/from a pickle file

In [5]:
def store_dict_pickle(file_name, dictionary):
    """
        Saves a dictionary to a pickle file

        :param file_name: the name of the file the data will be stored in
        :type file_name: str

        :param dictionary: the dictionary to be stored in the file
        :type dictionary: dict
    """

    with open('data/' + file_name + '.pkl', 'wb') as f:
        pickle.dump(dictionary, f, pickle.HIGHEST_PROTOCOL)


def load_dict_pickle(file_name):
    """
    Retrieves a dictionary form the specified file

    :param file_name: the name of the file to load
    :type file_name: str

    :return: the dictionary retrieved from the file
    :rtype dict
    """

    with open('data/' + file_name + '.pkl', 'rb') as f:
        dictionary =  pickle.load(f)
    return dictionary

# Loading Artist-Country Data

In [4]:
#intializing dictionary variables
dict_06 = load_dict_pickle("200_country_artist")
dict_07 = load_dict_pickle("200_country_artist")
dict_08 = load_dict_pickle("200_country_artist")
dict_09 = load_dict_pickle("200_country_artist")
dict_10 = load_dict_pickle("200_country_artist")
dict_11 = 
dict_12 = 
dict_13 = 

SyntaxError: unexpected EOF while parsing (<ipython-input-4-4a78dc645886>, line 16)